Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
    - speedup
    - eficiencia
 - output: 
    - p_cores


In [1]:
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

In [2]:
df_clean = pd.read_csv("dataset/clean_data.csv")

# df_clean['cores'] = df_clean['cores']/df_clean['cores'].max()
df_clean['input'] = df_clean['input']/df_clean['input'].max()

In [5]:
df_clean.columns

Index(['cores', 'input', 'average_runtime', 'speedup', 'efficiency', 'mem_ram',
       'freq_mem', 'freq_cpu', 'mem_l3'],
      dtype='object')

In [6]:
X = df_clean[['cores', 'input', 'average_runtime', 'speedup', 'mem_ram', 'freq_mem', 'freq_cpu', 'mem_l3']].values
y = df_clean['efficiency'].values

### Regression Models

#### MLP Regression Model

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [10]:
learning_rate = 0.00001
input_shape = (X_train.shape[1],)

# callback = EarlyStopping(monitor="loss", patience=20)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape = input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, batch_size=32, epochs=1000, verbose=2, validation_data=(X_test, y_test))

Epoch 1/1000
52/52 - 0s - loss: 1582.7279 - val_loss: 869.5767 - 406ms/epoch - 8ms/step
Epoch 2/1000
52/52 - 0s - loss: 425.4656 - val_loss: 785.2279 - 53ms/epoch - 1ms/step
Epoch 3/1000
52/52 - 0s - loss: 266.0887 - val_loss: 684.5352 - 54ms/epoch - 1ms/step
Epoch 4/1000
52/52 - 0s - loss: 166.4797 - val_loss: 540.7611 - 53ms/epoch - 1ms/step
Epoch 5/1000
52/52 - 0s - loss: 108.3683 - val_loss: 540.7273 - 52ms/epoch - 1ms/step
Epoch 6/1000
52/52 - 0s - loss: 76.3565 - val_loss: 448.1200 - 52ms/epoch - 994us/step
Epoch 7/1000
52/52 - 0s - loss: 56.1810 - val_loss: 341.4222 - 52ms/epoch - 996us/step
Epoch 8/1000
52/52 - 0s - loss: 45.0422 - val_loss: 345.1341 - 52ms/epoch - 999us/step
Epoch 9/1000
52/52 - 0s - loss: 39.9187 - val_loss: 314.6663 - 52ms/epoch - 1ms/step
Epoch 10/1000
52/52 - 0s - loss: 36.8373 - val_loss: 288.3309 - 53ms/epoch - 1ms/step
Epoch 11/1000
52/52 - 0s - loss: 34.1607 - val_loss: 291.1670 - 51ms/epoch - 984us/step
Epoch 12/1000
52/52 - 0s - loss: 32.3429 - val_l

In [11]:
loss = model.evaluate(X_test, y_test)
print(f'Erro no conjunto de teste: {loss}')

predictions = model.predict(X_test)

for i in range(10):
    print(f'Valor Real: {y_test[i]}, Previsão: {predictions[i][0]}')

23/23 [==============================] - 0s 545us/step - loss: 150.0966
Erro no conjunto de teste: 150.0966033935547
23/23 [==============================] - 0s 500us/step
Valor Real: 0.1421652660989802, Previsão: 1.0670876502990723
Valor Real: 0.1909549526707994, Previsão: 1.6698403358459473
Valor Real: 0.1362309885647307, Previsão: 1.3693947792053223
Valor Real: 0.0688319593505949, Previsão: 0.915354311466217
Valor Real: 0.1343610885294778, Previsão: 1.7729973793029785
Valor Real: 0.1712759335286006, Previsão: 1.6670632362365723
Valor Real: 0.1254793398850434, Previsão: 0.6679254174232483
Valor Real: 0.1133989266295716, Previsão: 1.4538445472717285
Valor Real: 0.1366358050892372, Previsão: 1.354097843170166
Valor Real: 0.1045667369935413, Previsão: 0.826517641544342


#### MLP Regression Model with Leave-One-Out-Validation (LOOCV)

In [12]:
loo = LeaveOneOut()
mse_scores = []
# callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))


for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    learning_rate = 0.001
    input_shape = (X_train.shape[1],)

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=100, verbose=0)

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

print(f'Erro médio quadrático usando LOOCV: {min(mse_scores)}')

1/1 [==============================] - 0s 35ms/step
Erro médio quadrático usando LOOCV: 4.475726450791921e-08


In [13]:
PATH_MODEL_SAVE = "model/pascal_model.keras"

model.save(PATH_MODEL_SAVE)

In [17]:
x_test_new = [
    [3,0,41.52263116836548,0.36647263598411695,7.0,2133,2.3,40],
    [2,0,41.432292222976685,0.3672716927022557,7.0,2133,2.3,40],
    [1,0,42.19862198829651,0.36060201448017554,7.0,2133,2.3,40]
]

model.predict(x=x_test_new)

1/1 [==============================] - 0s 25ms/step


array([[0.39101958],
       [0.40505552],
       [0.41904163]], dtype=float32)